[Link](https://datafai.com/auto-arima-using-pyramid-arima-python-package/)

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=18,9

# Load specific forecasting tools
from statsmodels.tsa.arima_model import ARIMA,ARIMAResults
from statsmodels.tsa.seasonal import seasonal_decompose

# for determining (p,q) orders
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf 
# for determining ARIMA orders
from pmdarima import auto_arima 

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")



In [ ]:
! pip show pandas

In [ ]:
# Load datasets
df1 = pd.read_csv('./Data/airline_passengers.csv',index_col='Month',parse_dates=True)
df1.index.freq = 'MS'
df1.head()

In [ ]:
df1.plot();

In [ ]:
data_decompose = seasonal_decompose(df1['Thousands of Passengers'])
data_decompose.plot();

## Stationarity Test

In [ ]:
from pmdarima.arima import ADFTest
adf_test_autoarima = ADFTest(alpha=0.05); 
_, stationarity_bool = adf_test_autoarima.is_stationary(df1["Thousands of Passengers"])
print(f'Is Stationary? {stationarity_bool}')

In [ ]:
len(df1["Thousands of Passengers"])

In [ ]:
train, test = df1["Thousands of Passengers"][:132], df1["Thousands of Passengers"][132:]
print(len(test)); print(len(train));

In [ ]:
train.plot()
test.plot();

## Use the best ARIMA model params to make predictions about the Test data

In [ ]:
arima_model = auto_arima(train, seasonal=True, m=12, trace=True)
arima_model.summary()

In [ ]:
predictions = pd.DataFrame(arima_model.predict(n_periods=3),index=test.index)
predictions.columns = ['Predicted_Passengers']


In [ ]:
plt.plot(test,label="Test")
plt.plot(predictions,label="Predicted_Pasengers")
plt.legend(loc="upper left")
plt.show();

## Evaluation

In [ ]:
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import r2_score, mean_squared_error

rmse_error = np.sqrt(mean_squared_error(test, predictions))
print(f'RMSE Error: {rmse_error}')
print("Test's mean:",test.mean())


r2_result = r2_score(test, predictions)
print(f"RSq: {r2_result}")